# Local Gradient Based Optimization
Notes copied from Marcels presentation

## Global methods with local gradient information

Several algorithms combine a global approach (to avoid getting stuck in local minima) with a local gradient approach (to quickly find the best values in a certain region), e.g. the "basin hopping" algorithm.

### Local gradient
For an ODE system of variables $\mathbf{x}$ with parameters $\mathbf{p}$ that we are interested in:
$$
\frac{\mathrm{d}\mathbf{x}(t; \mathbf{p})}{\mathrm{d}t} = f(\mathbf{x}, t; \mathbf{p}) 
$$

we want to know how the solution changes with changes in the parameters:
$$
\frac{\delta\mathbf{x}(t; \mathbf{p})}{\delta\mathbf{p}}
$$

This is called the *local sensitivity*.

*Note:* For optimization, we are in general only interested in the gradient of the error function. 

### How to get local gradient?

**Numerically** (often default in optimiztion algorithms): finite difference approximation

For each parameter $p_i$:
$$
\frac{\delta\mathbf{x}(t; \mathbf{p})}{\delta p_i} \approx \frac{\mathbf{x}(t; \mathbf{p} + \mathbf{e_i}\Delta p_i) - \mathbf{x}(t; \mathbf{p})}{\Delta p_i}
$$

This is problematic for ODEs:
* Difficult to chose $p_i$
* Numerical errors in the numerical integration of $\mathbf{x}(t; \mathbf{p})$ add up

Potentially better approach: *local forward sensitivity analysis*

Extend ODE system to include new variables:
$$
S_{ij} :=  \frac{\delta x_i}{\delta p_j}
$$

(from now on using usual short-hand notation, e.g. $x_i$ instead of $x_i(t; \mathbf{p})$)

Evolution of these variables as ODEs:
$$
\frac{\mathrm{d}\mathbf{S}_{j}}{\mathrm{d}t} = \mathbf{J}\,\mathbf{S}_{j} + \mathbf{F}_j,
$$

where: 
$$
\mathbf{J} = 
\begin{pmatrix}
\frac{\delta f_1}{\delta x_1} & \frac{\delta f_1}{\delta x_2} & \cdots & \frac{\delta f_1}{\delta x_n}\\
\frac{\delta f_2}{\delta x_1} & \frac{\delta f_2}{\delta x_2} & \cdots & \frac{\delta f_2}{\delta x_n}\\
\vdots & \vdots & \ddots & \vdots\\
\frac{\delta f_n}{\delta x_1} & \frac{\delta f_n}{\delta x_2} & \cdots & \frac{\delta f_n}{\delta x_n}\\
\end{pmatrix}, \quad
\mathbf{F}_j = 
\begin{pmatrix}
\frac{\delta f_1}{\delta p_j}\\
\frac{\delta f_2}{\delta p_j}\\
\vdots\\
\frac{\delta f_n}{\delta p_j}\\
\end{pmatrix}
$$

We can then solve the full system with the usual numerical integration algorithms.

In [1]:
def get_sensitivity_equations(group, parameters):
    eqs = group.equations
    diff_eqs = eqs.get_substituted_expressions(group.variables)
    diff_eq_names = [name for name, _ in diff_eqs]

    system = Matrix([diff_eq[1] for diff_eq in diff_eqs])
    J = system.jacobian(diff_eq_names)

    sensitivity = []
    sensitivity_names = []
    for parameter in parameters:
        F = system.jacobian([parameter])
        names = ['S_{}_{}'.format(diff_eq_name, parameter)
                 for diff_eq_name in diff_eq_names]
        sensitivity.append(J * Matrix(names) + F)
        sensitivity_names.append(names)

    new_eqs = []
    for names, sensitivity_eqs, param in zip(sensitivity_names, sensitivity, parameters):
        for name, eq, orig_var in zip(names, sensitivity_eqs, diff_eq_names):
            unit = eqs[orig_var].dim / group.namespace[parameter].dim
            new_eqs.append('d{lhs}/dt = {rhs} : {unit}'.format(lhs=name,
                                                           rhs=eq,
                                                           unit=repr(unit)))
    new_eqs = Equations('\n'.join(new_eqs))
    return new_eqs